# Descrição do programa

## Visão Geral

Este script realiza uma análise de risco para um fundo de investimento composto por ações listadas na B3. Ele automatiza a coleta de dados históricos de preços utilizando a API do Yahoo Finance, abrange um período de cinco anos, e calcula indicadores financeiros e de risco relevantes, como volatilidade, Value at Risk (VaR), Conditional Value at Risk (CVaR), beta via regressão, e índices de Sharpe e Treynor. O código também permite avaliar o risco sob múltiplos níveis de confiança, proporcionando uma visão abrangente da exposição do fundo a diferentes cenários de mercado.

## Autor

João Vitor Figueiredo Villa -[LinkedIn](https://www.linkedin.com/in/jo%C3%A3o-vitor-figueiredo-villa-8323b8323/)

# Instalação e Implementação dos Pacotes

In [2]:
%pip install pandas
%pip install numpy
%pip install statsmodels
%pip install yfinance
%pip install scipy
%pip install python-bcb
%pip install datetime
%pip install python-dateutil
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [3]:
import pandas as pd
import numpy as np
import statsmodels
from scipy.stats import norm
from datetime import datetime
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm

from bcb import sgs
import yfinance as yf

# Dados Iniciais para análise

Importando arquivo das ações

In [6]:
#resetar variáveis que serão usadas em loop
def resetV():
    global i, n, t, j
    i = n = t = j = 0
    return True

# Varíaveis de data que permite, que o código tenha series temporais fluídas e que se atualizam sempre que o código é rodado
agora = datetime.now()
dateFut = agora - relativedelta(years=5)

agora = agora.strftime("%Y-%m-%d")
dateFut = dateFut.strftime("%Y-%m-%d")

lTickers = ["PETR4.SA","JBSS3.SA","BBAS3.SA","VIVT3.SA","ITUB4.SA"]
listaAcoes = []
resetV()
for i in lTickers:
    acao = yf.Ticker(i).history(start=dateFut,end=agora)
    listaAcoes.append(acao)

Variáveis Básicas

In [10]:
#retornos logarítmicos diários
lRetD = []
#variância dos retornos logarítmicos diários
lVarc = []
resetV()
for i in listaAcoes:
    close_prices = i['Close']
    returns = np.log(close_prices / close_prices.shift(1)).dropna()
    
    lRetD.append(returns)
    lVarc.append(np.var(returns))

#z1 é para um nível de confiança de 95% (Primeira análise)
a1 = 0.95
z1 = norm.ppf(a1)
dz1 = norm.pdf(z1)

#z2 é para um nível de confiança de 99% (Segunda análise)
a2 = 0.99
z2 = norm.ppf(a2)
dz2 = norm.pdf(z2)

valInv = 1000000
lPorc = [0.2,0.2,0.20,0.20,0.20]
lInv = []
for i1 in lPorc:
    lInv.append(i1*valInv)

# 1) Análise Individual dos Investimentos

## Volatilidade

- Medida de variabilidade dos retornos de um ativo e é relacionado a grandes movimentos de preço, tanto para baixo quanto para cima da média

-  quanto maior, mais arriscado é o ativo

- Volatilidade é uma medida de risco total de um ativo ou portfólio (específico e sistemático)


In [11]:
#quanto menor a volatilidade, menor é a oscilação do ativo
lDP = []
resetV()
for i in lVarc:
    dp = np.sqrt(i)
    lDP.append(dp)

print("Volatilidade dos ativos:")
resetV()
for n,i in enumerate(lDP):
    print(f"{lTickers[n]} -> {i:.4f}")
print("-="*20)

resetV()
acoesRD = pd.concat([pd.Series(i, name=f'{lTickers[n]}') for n, i in enumerate(lRetD)], axis=1)

pesos = np.array(lPorc)
matrixCov = acoesRD.cov()

volPortf = np.sqrt(np.dot(pesos.T, np.dot(matrixCov, pesos)))
print(f"Volatilidade do Portfólio: {volPortf:.4f}")



Volatilidade dos ativos:
PETR4.SA -> 0.0231
JBSS3.SA -> 0.0207
BBAS3.SA -> 0.0179
VIVT3.SA -> 0.0144
ITUB4.SA -> 0.0178
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Volatilidade do Portfólio: 0.0123


## Value at Risk (VaR)

- Medida de risco de perda de capital

- Mede o quanto algum investimento pode perder, dado um certo nível de probabilidade em um período de tempo específico

- a estimar o quanto de recursos devem possuir para cobrir potenciais perdas

### Para 95% de nível de confiança

In [12]:

# a perda não ultrapassa o valor do VaR, a perda vai ser em 95% das vezes menor ou igual a esse valor 
# valor limite de perdas

lVaR1 = []
resetV()
for n,i in enumerate(lDP):
    valAtR = z1*i*lInv[n]
    lVaR1.append(valAtR)

print("VaR (Para 95% de nível de confiança):")
resetV()
for n,i in enumerate(lVaR1):
    print(f"Ação {n} -> perda máxima esperada de R$ {i:.2f} (Inv inicial = R$ {lInv[n]:.2f})")
print("-="*20)

print(f"Perda máxima do portfólio: R$ {np.sum(lVaR1):.2f}")

VaR (Para 95% de nível de confiança):
Ação 0 -> perda máxima esperada de R$ 7586.65 (Inv inicial = R$ 200000.00)
Ação 1 -> perda máxima esperada de R$ 6823.28 (Inv inicial = R$ 200000.00)
Ação 2 -> perda máxima esperada de R$ 5903.45 (Inv inicial = R$ 200000.00)
Ação 3 -> perda máxima esperada de R$ 4733.22 (Inv inicial = R$ 200000.00)
Ação 4 -> perda máxima esperada de R$ 5846.17 (Inv inicial = R$ 200000.00)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Perda máxima do portfólio: R$ 30892.77


### Para 99% de nível de confiança

In [13]:
# a perda não ultrapassa o valor do VaR, a perda vai ser em 99% das vezes menor ou igual a esse valor (cenário de incerteza)

lVaR2 = []
resetV()
for n,i in enumerate(lDP):
    valAtR = z2*i*lInv[n]
    lVaR2.append(valAtR)

print("VaR (Para 99% de nível de confiança):")
resetV()
for n,i in enumerate(lVaR2):
    print(f"Ação {n} -> perda máxima esperada de R${i:.2f} (Inv inicial = R$ {lInv[n]:.2f})")
print("-="*20)

print(f"Perda máxima esperada do portfólio: R$ {np.sum(lVaR2):.2f}")

VaR (Para 99% de nível de confiança):
Ação 0 -> perda máxima esperada de R$10729.94 (Inv inicial = R$ 200000.00)
Ação 1 -> perda máxima esperada de R$9650.30 (Inv inicial = R$ 200000.00)
Ação 2 -> perda máxima esperada de R$8349.37 (Inv inicial = R$ 200000.00)
Ação 3 -> perda máxima esperada de R$6694.29 (Inv inicial = R$ 200000.00)
Ação 4 -> perda máxima esperada de R$8268.35 (Inv inicial = R$ 200000.00)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Perda máxima esperada do portfólio: R$ 43692.24


## Máximo Drawdown

- Medida de risco de perda de capital, mas R é apenas uma medida histórica de o quanto de ruína um investimento já apresentou

- O MDD é a maior perda ocorrida desde um ponto de alta (pico) até um ponto de mínima em uma série histórica, ou seja, que ocorreu no passado

- Usado como métrica para reduzir o risco do portfólio

In [15]:
drawdownA = []
resetV()
for i in listaAcoes:
    precos = pd.Series(i['Close'])
    pico = precos.cummax()
    dd = (precos - pico) / pico
    drawdownA.append(dd.min())

print("Máximo Drawdown de cada ativo:")
resetV()
for n,i in enumerate(drawdownA):
    print(f"{lTickers[n]} -> {i:.2%}")

Máximo Drawdown de cada ativo:
PETR4.SA -> -39.10%
JBSS3.SA -> -55.78%
BBAS3.SA -> -28.91%
VIVT3.SA -> -31.32%
ITUB4.SA -> -36.17%


## Conditional Value at Risk (CVaR)

- Valores da cauda de distribuição de perdas em cenários de perda extrema (tende a ser maior do que o VaR)

- Mede a média das piores perdas além do VaR

### Para 95% de confiança

In [16]:
# média das perdas que ocorrem nos 5% piores cenários 

lCVaR1 = []
resetV()
for t,i in zip(lVaR1,lDP):
    cvar = i + (t*dz1/(1-a1))
    lCVaR1.append(cvar)

print("CVaR (Para 95% de nível de confiança):")
resetV()
for n, i in enumerate(lCVaR1):
    print(f"{lTickers[n]} -> pior média de perdas esperada de R$ {i:.2f} (VaR = R$ {lVaR1[n]:.2f})")
print("-="*20)

print(f"Pior média de perdas do portfólio esperado: R$ {np.sum(lCVaR1):.2f}")
    

CVaR (Para 95% de nível de confiança):
PETR4.SA -> pior média de perdas esperada de R$ 15649.10 (VaR = R$ 7586.65)
JBSS3.SA -> pior média de perdas esperada de R$ 14074.49 (VaR = R$ 6823.28)
BBAS3.SA -> pior média de perdas esperada de R$ 12177.14 (VaR = R$ 5903.45)
VIVT3.SA -> pior média de perdas esperada de R$ 9763.30 (VaR = R$ 4733.22)
ITUB4.SA -> pior média de perdas esperada de R$ 12058.98 (VaR = R$ 5846.17)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Pior média de perdas do portfólio esperado: R$ 63723.01


### Para 99% de confiança

In [17]:
# média das perdas que ocorrem nos 1% piores cenários

lCVaR2 = []
resetV()
for i,t in zip(lVaR2,lDP):
    cvar = i + (t*dz2/(1-a2))
    lCVaR2.append(cvar)

print("VaR (Para 99% de nível de confiança):")
resetV()
for n,i in enumerate(lCVaR2):
    print(f"{lTickers[n]} -> pior média de perdas de R$ {i:.2f} (VaR = R$ {lVaR2[n]:.2f})")
print("-="*20)

print(f"Pior média de perdas do portfólio: R$ {np.sum(lCVaR2):.2f}")

VaR (Para 99% de nível de confiança):
PETR4.SA -> pior média de perdas de R$ 10730.00 (VaR = R$ 10729.94)
JBSS3.SA -> pior média de perdas de R$ 9650.35 (VaR = R$ 9650.30)
BBAS3.SA -> pior média de perdas de R$ 8349.41 (VaR = R$ 8349.37)
VIVT3.SA -> pior média de perdas de R$ 6694.33 (VaR = R$ 6694.29)
ITUB4.SA -> pior média de perdas de R$ 8268.39 (VaR = R$ 8268.35)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Pior média de perdas do portfólio: R$ 43692.49


# 2) Análise Comparativa

## Correlação entre os ativos

Caracteristicas
* Medida como duas variáveis ou ativos se comportam entre si e do quão forte é a sua associção

* É estática no tempo


Interpretação do resultado
- Quanto mais próximo de zero, mais fraca é a correlação entre os ativos

- Quanto mais próximo de 1 ou -1, mais  forte é a correlação entre os ativos

In [21]:
print("Correlação entre os investimentos do portfólio:")
resetV()
for (i, t), valor in np.ndenumerate(matrixCov):
    if i < t:
        acaoI = lTickers[i]
        acaoT = lTickers[t]
        corrIxT = (matrixCov.iloc[i,t])/(lDP[i]*lDP[t])
        print(f"{acaoI} x {acaoT} = {corrIxT:.6f}")
    

Correlação entre os investimentos do portfólio:
PETR4.SA x JBSS3.SA = 0.126957
PETR4.SA x BBAS3.SA = 0.491892
PETR4.SA x VIVT3.SA = 0.115232
PETR4.SA x ITUB4.SA = 0.388511
JBSS3.SA x BBAS3.SA = 0.177483
JBSS3.SA x VIVT3.SA = 0.158855
JBSS3.SA x ITUB4.SA = 0.165555
BBAS3.SA x VIVT3.SA = 0.301951
BBAS3.SA x ITUB4.SA = 0.643246
VIVT3.SA x ITUB4.SA = 0.304851


## Correlação entre Fundo e Mercado

Um tipo de beta que mede a sensibilidade do fundo ao mercado, normalizando a covariância pelo risco do mercado (variância dos retornos do benchmark)

In [20]:
retornoFundoDia = pd.Series(acoesRD.dot(pesos), index=acoesRD.index)

ibov5y = yf.Ticker("^BVSP").history(start=dateFut,end=agora)
ibov5y = ibov5y['Close']
retIbovDia = np.log(ibov5y / ibov5y.shift(1)).dropna()

corrMF = np.corrcoef(retornoFundoDia, retIbovDia)[0,1]
print(f"Covariância entre fundo e mercado: {corrMF:.6f}")

Covariância entre fundo e mercado: 0.819312


## Indice de Sharpe

Interpretação do resultado
- Sharpe = 0: Retorno abaixo da tx livre de risco (mau desempenho ajustado ao risco)

- 0 < Sharpe < 1: Retorno baixo em relação ao risco

- 1 < Sharpe < 1.99: Bom (retorno razoável para o rísco assumido)

- 2 < Sharpe < 2.99: Muit Bom (portfólio eficiente)

- Sharpe > 3: Excelente (alto retorno com risco controlado)

In [ ]:
cdiAnual = sgs.get(1178,last=1)
cdiAnual = (cdiAnual.iloc[0,0])/100

cdiDia = (1 + cdiAnual)**(1/252) - 1

mediaRetDia = np.dot(acoesRD.mean(), pesos)

sharpeF =  (mediaRetDia - cdiDia)/ volPortf
print(f"Índice de Sharpe do portfólio: {sharpeF:.4f}")

Índice de Sharpe do portfólio: 0.0146


## Beta e Alpha do fundo

O beta é a proporção que o fundo se movimenta com o benckmark (Ibovespa)

Interpretação do resultado do alpha
- Alpha > 0: portfólio está gerando retorno acima do esperado pelo CAPM

- Alpha < 0: portfólio está gerando retorno abaixo do esperado pelo CAPM

Interpretação do resultado do beta
- Beta < 0: Inversamente correlacionado (o fundo anda no sentido oposto ao mercado)

- Beta = 0: Independente do mercado (o fundo não se movimenta com o mercado)

- Beta entre 0 e 1: Menos volátil que o mercado (defensivo, segue o mercado com menos intensidade)

- Beta = 1: Neutro ao mercado (o fundo sobe ou cai na mesma proporção do benchmark)

- Beta > 1: Mais volátil que o mercado (agressivo, amplifica os movimentos do mercado)

In [ ]:
medIbovDia = np.mean(retIbovDia)

dfAcoesBt = pd.merge(retIbovDia.to_frame('ibov'), 
                     retornoFundoDia.to_frame('fundo'), 
                     left_index=True, right_index=True, 
                     how='inner')
dfAcoesBt = dfAcoesBt.dropna()

X = sm.add_constant(dfAcoesBt['ibov'])
y = dfAcoesBt['fundo']

modeloRL = sm.OLS(y, X).fit()
print(modeloRL)

beta = modeloRL.params['ibov']
alpha = modeloRL.params['const']

print("-="*20)
print(f"Beta: {beta:.4f}")
print(f"Alpha: {alpha:.4f}")



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Beta: 0.8488
Alpha: 0.0004


## Indice de Treynor

No calculo do Indice de Treynor foi usado o retorno do CDI (atualizado sempre que o código for rodado) como taxa livre de risco

Interpretação do resultado
- Positivo e alto: O fundo está entregando bom retorno ajustado ao risco de mercado (quanto maior, melhor)
- Próximo de zero: O fundo não gera retorno além da taxa livre de risco, mesmo assumindo risco de mercado
- Negativo: O fundo teve desempenho pior que a taxa livre de risco, mesmo assumindo risco (performance ruim)

In [32]:
treynorD =  (mediaRetDia - cdiDia)/beta
print(f"Índice de Treynor diário: {treynorD:.6f}")

retFundoAn = (1+mediaRetDia)**252 - 1
treynorA = (retFundoAn - cdiAnual)/beta
print(f"Índice de Treynor anual: {treynorA:.6f}")

Índice de Treynor diário: 0.000212
Índice de Treynor anual: 0.062599


## Retorno Esperado (CAPM)

O ativo livre de risco será o CDI e o benckmark se manterá o Ibovespa

In [39]:
retIbovAn = (1+medIbovDia)**252 - 1

capm = cdiAnual + beta*(retIbovAn - cdiAnual)
print(f"Retorno  minimo esperado do portfólio: {capm:.2%}")

Retorno  minimo esperado do portfólio: 10.74%


## Tracking Error

Caracteristicas
- desvio padrão entre os retornos diferenciais entre o fundo e o benchmark (Ibovespa)

- mede a consistência do fundo em seguir o benchmark

- quanto menor, mais o fundo se comporta como o benchmark (caso de fundos passivos) e quanto maior, mais o fundo se descola do benchmark (típico de fundos ativos)

Interpreteação do resultado anualizado

- 0% < TE < 2%: muito baixo - fundo passivo, segue de perto o índice

- 2% < TE < 4%: moderado - fundo ativo com alguma liberdade

- 4% < TE < 7%: alto - fundo ativo com forte autonomia

- TE > 7%: muito alto - fundo com estratégias muito diferentes do benchmark

In [22]:
trackingErD = np.std(retornoFundoDia - retIbovDia, ddof=1)
trackingErA = trackingErD * np.sqrt(252)

print(f"Tracking Error diário: {trackingErD:.4%}")
print(f"Tracking Error anualizado: {trackingErA:.4%}")

Tracking Error diário: 0.7296%
Tracking Error anualizado: 11.5824%
